# Example of Training with MotionSense

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
import pandas as pd
from pathlib import Path

sys.path.append("..")

## Read Processed Data

In [23]:
from librep.utils.dataset import PandasDatasetsIO

In [24]:
dataset_path = Path("../data/views/MotionSense/balanced_view")
train, validation, test = PandasDatasetsIO(dataset_path).load()

In [25]:
train = pd.concat([train, validation], ignore_index=True)
train

,Unnamed: 0,attitude.roll-0,attitude.roll-1,attitude.roll-2,attitude.roll-3,attitude.roll-4,attitude.roll-5,attitude.roll-6,attitude.roll-7,attitude.roll-8,...,userAcceleration.z-295,userAcceleration.z-296,userAcceleration.z-297,userAcceleration.z-298,userAcceleration.z-299,activity code,length,trial_code,index,user
0,0,1.093387,1.129419,1.166286,1.176539,1.218638,1.218076,1.169095,1.104227,1.041327,...,0.025803,0.015770,0.109555,0.172113,0.248941,0,300,1,1200,16
1,1,1.528132,1.527992,1.527765,1.516768,1.493941,1.476302,1.455153,1.441702,1.443440,...,0.578416,0.572674,0.490945,0.345010,0.129649,0,300,1,0,1
2,2,-1.331274,-1.315688,-1.315034,-1.315621,-1.287623,-1.218232,-1.139884,-1.054611,-0.970854,...,0.102089,0.124263,0.122183,0.125418,0.230901,0,300,2,600,21
3,3,0.715184,0.679044,0.662668,0.662928,0.670537,0.666622,0.652839,0.662879,0.723967,...,-0.033105,0.119528,0.057709,-0.057865,-0.097547,0,300,11,0,7
4,4,-2.004835,-2.041697,-2.044302,-2.038474,-2.055756,-2.102689,-2.158746,-2.202283,-2.232511,...,0.072444,0.005214,-0.231522,-0.560348,-0.678777,0,300,2,1800,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909,325,-0.809114,-0.504596,-0.200415,-0.069472,-0.009363,-0.019789,-0.084377,-0.162347,-0.184036,...,-0.754819,-0.570919,-1.006368,-1.232031,-0.321935,5,300,16,0,6
1910,326,1.225254,1.219554,1.262265,1.213586,1.229215,1.309294,1.371141,1.403459,1.491145,...,1.737598,1.105376,-0.184008,-1.426659,-1.711004,5,300,9,4500,2
1911,327,3.138105,-3.105622,-3.073557,-3.034694,-2.983038,-2.938967,-2.891639,-2.733605,-1.150408,...,-1.048692,-0.806513,-0.533924,1.383309,-0.669395,5,300,9,0,6
1912,328,1.476984,1.582842,1.692089,1.797948,1.852740,1.886674,1.885654,1.810233,1.708376,...,-0.326163,-0.657696,-0.787899,0.421130,0.055916,5,300,9,600,2


## Create the MotionSense Dataset

In [26]:
from librep.datasets.multimodal import PandasMultiModalDataset

In [27]:
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

train_dataset = PandasMultiModalDataset(
    train,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)
validation_dataset = PandasMultiModalDataset(
    validation,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)
test_dataset = PandasMultiModalDataset(
    test,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

In [28]:
train_dataset[0]

(array([-0.120843,  0.470622,  0.269861, ..., -0.046833, -0.283947,
        -0.521564]),
 0)

In [29]:
train_dataset.window_slices, train_dataset.window_names, train_dataset.num_windows

([(0, 300), (300, 600), (600, 900), (900, 1200), (1200, 1500), (1500, 1800)],
 ['userAcceleration.x',
  'userAcceleration.y',
  'userAcceleration.z',
  'rotationRate.x',
  'rotationRate.y',
  'rotationRate.z'],
 6)

In [30]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [31]:
fft_transform = FFT()
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")
train_dataset_fft = transformer(train_dataset)
validation_dataset_fft = transformer(validation_dataset)
test_dataset_fft = transformer(test_dataset)
train_dataset_fft.window_slices, train_dataset_fft.window_names, train_dataset_fft.num_windows

([(0, 300), (300, 600), (600, 900), (900, 1200), (1200, 1500), (1500, 1800)],
 ['fft.userAcceleration.x',
  'fft.userAcceleration.y',
  'fft.userAcceleration.z',
  'fft.rotationRate.x',
  'fft.rotationRate.y',
  'fft.rotationRate.z'],
 6)

In [32]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT
from librep.transforms.stats import StatsTransform

fft_transform = FFT()
stats_transform = StatsTransform()
transformer = TransformMultiModalDataset(transforms=[fft_transform, stats_transform], new_window_name_prefix="fft-stats.")
train_dataset_fft_stats = transformer(train_dataset)
validation_dataset_fft_stats = transformer(validation_dataset)
test_dataset_fft_stats = transformer(test_dataset)
train_dataset_fft_stats.window_slices, train_dataset_fft_stats.window_names, train_dataset_fft_stats.num_windows

([(0, 19), (19, 38), (38, 57), (57, 76), (76, 95), (95, 114)],
 ['fft-stats.userAcceleration.x',
  'fft-stats.userAcceleration.y',
  'fft-stats.userAcceleration.z',
  'fft-stats.rotationRate.x',
  'fft-stats.rotationRate.y',
  'fft-stats.rotationRate.z'],
 6)

In [34]:
from librep.datasets.multimodal import combine_multi_modal_datasets
combined_train_dset = combine_multi_modal_datasets(train_dataset_fft, train_dataset_fft_stats)
combined_test_dset = combine_multi_modal_datasets(test_dataset_fft, test_dataset_fft_stats)
combined_train_dset.window_slices, combined_train_dset.window_names, combined_train_dset.num_windows

([(0, 300),
  (300, 600),
  (600, 900),
  (900, 1200),
  (1200, 1500),
  (1500, 1800),
  (1800, 1819),
  (1819, 1838),
  (1838, 1857),
  (1857, 1876),
  (1876, 1895),
  (1895, 1914)],
 ['fft.userAcceleration.x',
  'fft.userAcceleration.y',
  'fft.userAcceleration.z',
  'fft.rotationRate.x',
  'fft.rotationRate.y',
  'fft.rotationRate.z',
  'fft-stats.userAcceleration.x',
  'fft-stats.userAcceleration.y',
  'fft-stats.userAcceleration.z',
  'fft-stats.rotationRate.x',
  'fft-stats.rotationRate.y',
  'fft-stats.rotationRate.z'],
 12)

In [35]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT
from librep.transforms.resampler import SimpleResampler
from librep.transforms.stats import StatsTransform

resampler = SimpleResampler(new_sample_size=100)
fft_transform = FFT()
stats_transform = StatsTransform()
transformer = TransformMultiModalDataset(transforms=[resampler, fft_transform], new_window_name_prefix="resample-100-fft.")
resampled_train_dataset_fft_stats = transformer(train_dataset)
resampled_validation_dataset_fft_stats = transformer(validation_dataset)
resampled_test_dataset_fft_stats = transformer(test_dataset)
resampled_train_dataset_fft_stats.window_slices, resampled_train_dataset_fft_stats.window_names, resampled_train_dataset_fft_stats.num_windows

([(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)],
 ['resample-100-fft.userAcceleration.x',
  'resample-100-fft.userAcceleration.y',
  'resample-100-fft.userAcceleration.z',
  'resample-100-fft.rotationRate.x',
  'resample-100-fft.rotationRate.y',
  'resample-100-fft.rotationRate.z'],
 6)

## Train

In [36]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [37]:
result = multi_run_experiment(train_dataset, [test_dataset, validation_dataset, test_dataset])
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661519640.8258512
    result:
    -   accuracy: 0.8028455284552846
        f1 score (macro): 0.8001367810489692
        f1 score (micro): 0.8028455284552846
        f1 score (weighted): 0.8055542758616
    -   accuracy: 1.0
        f1 score (macro): 1.0
        f1 score (micro): 1.0
        f1 score (weighted): 1.0
    -   accuracy: 0.8028455284552846
        f1 score (macro): 0.8001367810489692
        f1 score (micro): 0.8028455284552846
        f1 score (weighted): 0.8055542758616
    run id: 1
    start: 1661519637.0573373
    time taken: 3.7685139179229736
-   end: 1661519644.7929575
    result:
    -   accuracy: 0.8130081300813008
        f1 score (macro): 0.8083405172585767
        f1 score (micro): 0.8130081300813008
        f1 score (weighted): 0.8176757429040249
    -   accuracy: 1.0
        f1 score (macro): 1.0
        f1 score (micro): 1.0
        f1 score (weighted): 1.0
    -   accuracy: 0.8130081300813008
        f1 score (macro): 0.8083405172585767
    

In [38]:
result = multi_run_experiment(train_dataset_fft, test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661519652.2399166
    result:
    -   accuracy: 0.8780487804878049
        f1 score (macro): 0.8787366742308159
        f1 score (micro): 0.8780487804878049
        f1 score (weighted): 0.8773608867447937
    run id: 1
    start: 1661519648.6803925
    time taken: 3.5595240592956543
-   end: 1661519655.661257
    result:
    -   accuracy: 0.8821138211382114
        f1 score (macro): 0.8823800752325783
        f1 score (micro): 0.8821138211382114
        f1 score (weighted): 0.8818475670438444
    run id: 2
    start: 1661519652.23992
    time taken: 3.421337127685547
-   end: 1661519659.0383623
    result:
    -   accuracy: 0.8821138211382114
        f1 score (macro): 0.8829159311206777
        f1 score (micro): 0.8821138211382114
        f1 score (weighted): 0.881311711155745
    run id: 3
    start: 1661519655.6612585
    time taken: 3.377103805541992



In [39]:
result = multi_run_experiment(train_dataset_fft_stats, test_dataset_fft_stats)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661519659.7032201
    result:
    -   accuracy: 0.8943089430894309
        f1 score (macro): 0.8937923706809404
        f1 score (micro): 0.8943089430894309
        f1 score (weighted): 0.8948255154979214
    run id: 1
    start: 1661519659.0696335
    time taken: 0.6335866451263428
-   end: 1661519660.3548586
    result:
    -   accuracy: 0.8841463414634146
        f1 score (macro): 0.8836762217260113
        f1 score (micro): 0.8841463414634146
        f1 score (weighted): 0.884616461200818
    run id: 2
    start: 1661519659.7032225
    time taken: 0.6516361236572266
-   end: 1661519660.970105
    result:
    -   accuracy: 0.8963414634146342
        f1 score (macro): 0.895699486716449
        f1 score (micro): 0.8963414634146342
        f1 score (weighted): 0.8969834401128193
    run id: 3
    start: 1661519660.3548605
    time taken: 0.6152443885803223



In [40]:
result = multi_run_experiment(combined_train_dset, combined_test_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661519664.3230143
    result:
    -   accuracy: 0.9186991869918699
        f1 score (macro): 0.9182317792570172
        f1 score (micro): 0.91869918699187
        f1 score (weighted): 0.9191665947267227
    run id: 1
    start: 1661519661.0026617
    time taken: 3.320352554321289
-   end: 1661519667.6212258
    result:
    -   accuracy: 0.9227642276422764
        f1 score (macro): 0.9225678216150697
        f1 score (micro): 0.9227642276422763
        f1 score (weighted): 0.9229606336694832
    run id: 2
    start: 1661519664.3230164
    time taken: 3.2982094287872314
-   end: 1661519670.9082303
    result:
    -   accuracy: 0.9065040650406504
        f1 score (macro): 0.9064962947204198
        f1 score (micro): 0.9065040650406504
        f1 score (weighted): 0.9065118353608811
    run id: 3
    start: 1661519667.6212282
    time taken: 3.2870020866394043



In [42]:
result = multi_run_experiment(resampled_train_dataset_fft_stats, resampled_test_dataset_fft_stats)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661519698.4826944
    result:
    -   accuracy: 0.8780487804878049
        f1 score (macro): 0.8793346813660081
        f1 score (micro): 0.8780487804878049
        f1 score (weighted): 0.8767628796096014
    run id: 1
    start: 1661519696.5960474
    time taken: 1.8866469860076904
-   end: 1661519700.4154677
    result:
    -   accuracy: 0.9065040650406504
        f1 score (macro): 0.9065941043776761
        f1 score (micro): 0.9065040650406504
        f1 score (weighted): 0.9064140257036247
    run id: 2
    start: 1661519698.4826968
    time taken: 1.9327709674835205
-   end: 1661519702.3362632
    result:
    -   accuracy: 0.8963414634146342
        f1 score (macro): 0.8968320728507485
        f1 score (micro): 0.8963414634146342
        f1 score (weighted): 0.8958508539785197
    run id: 3
    start: 1661519700.4154694
    time taken: 1.9207937717437744

